# Train Stacking

In [1]:
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        # Currently, there's no separate manual seed function for MPS like torch.cuda.manual_seed_all()
        pass 
    else:
        print("MPS device not available. Using CPU.")
        
set_seed()

Best F1 score BERT model on HS_C dataset are used

In [2]:
# This automates data tabulation onto google sheets 

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

import os

# new directory path
new_directory = '/Users/levan/ATENEO MASTERAL/Thesis'

# Change the current working directory
os.chdir(new_directory)

# Use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('thesis-432315-12daec8d1ff6.json', scope)

service = build('sheets', 'v4', credentials=creds)

client = gspread.authorize(creds)

spreadsheet_id = '13Fk5oXX9B_mdHmNpMKQMy29y9iiHWrgQCa4hUTiQKD0' 

## Load Dataset

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix, precision_score, recall_score, accuracy_score



import os

# new directory path
new_directory = '/Users/levan/ATENEO MASTERAL/Thesis/Development'

# Change the current working directory
os.chdir(new_directory)

#Load Data
df = pd.read_csv('Corpus/FiReCS/FiReCS_data_b.csv')
X = df['review'].values
y = df['label'].values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)



## Load Models and Tokenizers

In [4]:
def load_model_and_tokenizer(model_path, tokenizer_path, base_model):
    # Load the tokenizer from the local directory
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    # Load the configuration from the base model, then update configuration if needed
    config = AutoConfig.from_pretrained(base_model, num_labels=3)

    # Initialize the model with the configuration
    model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

    # Ensure the model is in evaluation mode
    model.eval()
    
    return model, tokenizer

model_info = {
    
    'BERT cased': {
        'model_path': 'BERT models/1 FiReCS/fire_bert-base-cased-finetuned',
        'tokenizer_path': 'BERT models/1 FiReCS/fire_bert-base-cased-finetuned',
        'base_model': 'google-bert/bert-base-cased'
    },

    'DistilBERT uncased': {
        'model_path': 'BERT models/1 FiReCS/fire_distilbert-base-uncased-finetuned',
        'tokenizer_path': 'BERT models/1 FiReCS/fire_distilbert-base-uncased-finetuned',
        'base_model': 'distilbert/distilbert-base-uncased'
    },

    'DeBERTa': {
        'model_path': 'BERT models/1 FiReCS/fire_DeBERTa-finetuned',
        'tokenizer_path': 'BERT models/1 FiReCS/fire_DeBERTa-finetuned',
        'base_model': 'microsoft/deberta-v3-base'
    },
    
    'BERT uncased': {
        'model_path': 'BERT models/1 FiReCS/fire_bert-base-uncased-finetuned',
        'tokenizer_path': 'BERT models/1 FiReCS/fire_bert-base-uncased-finetuned',
        'base_model': 'google-bert/bert-base-uncased'
    },
}


models_and_tokenizers = {name: load_model_and_tokenizer(info['model_path'], 
                                                        info['tokenizer_path'], 
                                                        info['base_model']) 
                         for name, info in model_info.items()}


## Apply Tokenization

In [5]:
class MultiModelTextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, models_and_tokenizers, max_len):
        self.texts = texts
        self.labels = labels
        self.models_and_tokenizers = models_and_tokenizers
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]

        model_inputs = {}
        for name, (model, tokenizer) in self.models_and_tokenizers.items():
            encoding = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=False,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt',
            )
            model_inputs[name] = {
                'input_ids': encoding['input_ids'].squeeze(0),  # Remove batch dimension
                'attention_mask': encoding['attention_mask'].squeeze(0)  # Remove batch dimension
            }
        return {'model_inputs': model_inputs, 'label': torch.tensor(label, dtype=torch.long)}


## BERT Training for Stacking

In [6]:
def train_base_learner(model, data_loader, optimizer, device):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = model.to(device)
    model.train()
    correct_predictions = 0
    losses = []

    for batch in data_loader:
        # Accessing inputs for the current model
        inputs = batch['model_inputs']
        labels = batch['label'].to(device)

        for model_name, model_inputs in inputs.items():
            # Forward pass
            outputs = model(input_ids=model_inputs['input_ids'].squeeze(1).to(device),
                attention_mask=model_inputs['attention_mask'].squeeze(1).to(device),
                labels=labels)

            loss = outputs.loss
            preds = outputs.logits.argmax(dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
            
            # Backward pass
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    return correct_predictions.float() / len(data_loader.dataset), np.mean(losses).astype(np.float32)




def eval_model(model, data_loader, device):
    model = model.eval()
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1)
            correct_predictions += torch.sum(preds == labels)

    return correct_predictions.double() / len(data_loader.dataset)


## Train Stacking

In [7]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from torch.utils.data import DataLoader
from transformers import AdamW

def train_stacking_classifier(models_and_tokenizers, X_train, y_train, device, max_len=128):
    n_splits = 4
    skf = StratifiedKFold(n_splits=n_splits)
    base_learners_predictions_train = np.zeros((len(X_train), len(models_and_tokenizers)))
    fold_idx = 0

    for train_idx, val_idx in skf.split(X_train, y_train):
        fold_idx += 1
        print(f"Training fold {fold_idx}/{n_splits}...")
        train_texts, val_texts = X_train[train_idx], X_train[val_idx]
        train_labels, val_labels = y_train[train_idx], y_train[val_idx]
        
        model_idx = 0
        for name, info in model_info.items():
            # Reload the model and tokenizer to reset weights
            model, tokenizer = load_model_and_tokenizer(info['model_path'], info['tokenizer_path'], info['base_model'])
            model.to(device)
            
            train_dataset = MultiModelTextDataset(train_texts, train_labels, {name: (model, tokenizer)}, max_len)
            val_dataset = MultiModelTextDataset(val_texts, val_labels, {name: (model, tokenizer)}, max_len)
            train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
            
            optimizer = AdamW(model.parameters(), lr=5e-5)
            train_base_learner(model, train_loader, optimizer, device)
            
            val_preds = []
            model.eval()
            for batch in val_loader:
                inputs = batch['model_inputs'][name]
                with torch.no_grad():
                    outputs = model(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
                val_preds.extend(outputs.logits.argmax(dim=1).cpu().numpy())
            
            base_learners_predictions_train[val_idx, model_idx] = np.array(val_preds)
            model_idx += 1

    return base_learners_predictions_train, y_train
    

def get_predictions(model_info, X, y, device, max_len=128):
    predictions = np.zeros((len(X), len(model_info)))
    model_idx = 0
    for name, info in model_info.items():
        # Reload the model and tokenizer
        model, tokenizer = load_model_and_tokenizer(info['model_path'], info['tokenizer_path'], info['base_model'])
        model.to(device)
        model.eval()
        
        dataset = MultiModelTextDataset(X, y, {name: (model, tokenizer)}, max_len)
        loader = DataLoader(dataset, batch_size=16, shuffle=False)
        
        preds = []
        for batch in loader:
            inputs = batch['model_inputs'][name]
            with torch.no_grad():
                outputs = model(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
            preds.extend(outputs.logits.argmax(dim=1).cpu().numpy())
        
        predictions[:, model_idx] = np.array(preds)
        model_idx += 1

    return predictions


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

## Perform Stacking

In [8]:
%load_ext memory_profiler

In [9]:
%%memit

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Train stacking classifier (using 4-fold cross validation for each BERT model) on Train Set and retrieve training meta-features
train_meta_features, _ = train_stacking_classifier(model_info, X_train, y_train, device)

# Generate predictions for the test set to be used as test meta-features
test_meta_features = get_predictions(model_info, X_test, y_test, device)

Training fold 1/4...


/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training fold 2/4...
Training fold 3/4...
Training fold 4/4...
peak memory: 6568.47 MiB, increment: 4220.16 MiB


In [10]:
print("Unique predictions from base learners on the test set:")
for i in range(test_meta_features.shape[1]):
    unique_preds = np.unique(test_meta_features[:, i])
    print(f"Base learner {i+1}: {unique_preds}")


Unique predictions from base learners on the test set:
Base learner 1: [0. 1. 2.]
Base learner 2: [0. 1. 2.]
Base learner 3: [0. 1. 2.]
Base learner 4: [0. 1. 2.]


In [11]:
import numpy as np
import pandas as pd

# Convert the meta-features to a DataFrame
df_test_meta = pd.DataFrame(test_meta_features, columns=[f"Base_{i+1}" for i in range(test_meta_features.shape[1])])

# Calculate the correlation matrix
correlation_matrix = df_test_meta.corr()
print("Correlation Matrix of Base Learner Predictions on the Test Set:")
print(correlation_matrix)

Correlation Matrix of Base Learner Predictions on the Test Set:
          Base_1    Base_2    Base_3    Base_4
Base_1  1.000000  0.852029  0.843146  0.862713
Base_2  0.852029  1.000000  0.843186  0.853686
Base_3  0.843146  0.843186  1.000000  0.848554
Base_4  0.862713  0.853686  0.848554  1.000000


In [12]:
# Count unique rows in the meta-features
unique_meta_features = np.unique(test_meta_features, axis=0)
num_unique_combinations = len(unique_meta_features)
num_samples = len(test_meta_features)
print(f"Number of unique meta-feature combinations: {num_unique_combinations} out of {num_samples} samples")

Number of unique meta-feature combinations: 55 out of 1416 samples


### Train LR

In [13]:
%%memit

lr_meta_classifier = LogisticRegression()
lr_meta_classifier.fit(train_meta_features, y_train)

lr_final_predictions = lr_meta_classifier.predict(test_meta_features)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


peak memory: 6210.91 MiB, increment: 0.59 MiB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Train NB

In [14]:
%%memit

nb_meta_classifier = GaussianNB()
nb_meta_classifier.fit(train_meta_features, y_train)

nb_final_predictions = nb_meta_classifier.predict(test_meta_features)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

peak memory: 6210.92 MiB, increment: 0.00 MiB


### Train DT

In [15]:
%%memit

dt_meta_classifier = DecisionTreeClassifier(max_depth=None)
dt_meta_classifier.fit(train_meta_features, y_train)

dt_final_predictions = dt_meta_classifier.predict(test_meta_features)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

peak memory: 6211.09 MiB, increment: 0.17 MiB


### Train SVM

In [16]:
%%memit

svm_meta_classifier = SVC(kernel='linear', probability=True)
svm_meta_classifier.fit(train_meta_features, y_train)

svm_final_predictions = svm_meta_classifier.predict(test_meta_features)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


peak memory: 6212.30 MiB, increment: 1.20 MiB


## EVALUATE MODELS

### LR

In [17]:
# Evaluate
precision = precision_score(y_test, lr_final_predictions, average='macro')
recall = recall_score(y_test, lr_final_predictions, average='macro')
accuracy = accuracy_score(y_test, lr_final_predictions)
f1 = f1_score(y_test, lr_final_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, lr_final_predictions)

print("Logistic Regression")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Ensemble accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


Logistic Regression
Precision: 0.8235245033543211
Recall: 0.8184910594217816
Ensemble accuracy: 0.8177966101694916
F1 Score: 0.8204395958329472
Confusion Matrix:
[[375  84   1]
 [ 62 383  46]
 [  4  61 400]]


In [18]:
probabilities = lr_meta_classifier.predict_proba(test_meta_features)

# Calculate precision-recall curve and find optimal thresholds for each class
lr_thresh = []
for i in range(probabilities.shape[1]):  # Iterate over each class
    precision, recall, thresholds = precision_recall_curve(y_test == i, probabilities[:, i])
    f1_scores = 2 * (precision * recall) / (precision + recall)
    optimal_idx = np.nanargmax(f1_scores)  # Handle cases where precision and recall are both zero
    lr_thresh.append(thresholds[optimal_idx])

# Apply the optimized thresholds to make adjusted predictions
adjusted_predictions = np.array([np.argmax([probabilities[i, j] if probabilities[i, j] >= lr_thresh[j] else 0
                                         for j in range(probabilities.shape[1])])
                              for i in range(len(probabilities))])

# Evaluate
adjusted_precision = precision_score(y_test, adjusted_predictions, average='macro')
adjusted_recall = recall_score(y_test, adjusted_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_test, adjusted_predictions)
adjusted_f1 = f1_score(y_test, adjusted_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, adjusted_predictions)

print("Optimized thresholds for each class:", lr_thresh)
print("Logistic Regression")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Ensemble accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1}")
print("Confusion Matrix:")
print(conf_matrix)

Optimized thresholds for each class: [0.25464822533832987, 0.5635565104377177, 0.30280071723613877]
Logistic Regression
Precision: 0.8285
Recall: 0.8268
Ensemble accuracy: 0.8256
F1 Score: 0.8275098616724273
Confusion Matrix:
[[392  67   1]
 [ 69 376  46]
 [  4  60 401]]


In [19]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!B4:E4'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


### NB

In [20]:
# Evaluate
precision = precision_score(y_test, nb_final_predictions, average='macro')
recall = recall_score(y_test, nb_final_predictions, average='macro')
accuracy = accuracy_score(y_test, nb_final_predictions)
f1 = f1_score(y_test, nb_final_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, nb_final_predictions)

print("Naive Bayes")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Ensemble accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


Naive Bayes
Precision: 0.8202543159072034
Recall: 0.8117103975660376
Ensemble accuracy: 0.8114406779661016
F1 Score: 0.814554625268911
Confusion Matrix:
[[362  97   1]
 [ 56 389  46]
 [  4  63 398]]


In [21]:
probabilities = nb_meta_classifier.predict_proba(test_meta_features)

# Calculate precision-recall curve and find optimal thresholds for each class
nb_thresh = []
for i in range(probabilities.shape[1]):  # Iterate over each class
    precision, recall, thresholds = precision_recall_curve(y_test == i, probabilities[:, i])
    f1_scores = 2 * (precision * recall) / (precision + recall)
    optimal_idx = np.nanargmax(f1_scores)  # Handle cases where precision and recall are both zero
    nb_thresh.append(thresholds[optimal_idx])

# Apply the optimized thresholds to make adjusted predictions
adjusted_predictions = np.array([np.argmax([probabilities[i, j] if probabilities[i, j] >= nb_thresh[j] else 0
                                         for j in range(probabilities.shape[1])])
                              for i in range(len(probabilities))])

# Evaluate
adjusted_precision = precision_score(y_test, adjusted_predictions, average='macro')
adjusted_recall = recall_score(y_test, adjusted_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_test, adjusted_predictions)
adjusted_f1 = f1_score(y_test, adjusted_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, adjusted_predictions)

print("Optimized thresholds for each class:", nb_thresh)
print("Naive Bayes")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Ensemble accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1}")
print("Confusion Matrix:")
print(conf_matrix)

Optimized thresholds for each class: [0.037746478555672584, 0.9263218803681593, 0.6535968125662914]
Naive Bayes
Precision: 0.8277
Recall: 0.8271
Ensemble accuracy: 0.8256
F1 Score: 0.8270965613094822
Confusion Matrix:
[[402  57   1]
 [ 76 369  46]
 [  4  63 398]]


In [22]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!B5:E5'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


### DT

In [23]:
# Evaluate
precision = precision_score(y_test, dt_final_predictions, average='macro')
recall = recall_score(y_test, dt_final_predictions, average='macro')
accuracy = accuracy_score(y_test, dt_final_predictions)
f1 = f1_score(y_test, dt_final_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, dt_final_predictions)

print("Decision Tree")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Ensemble accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


Decision Tree
Precision: 0.8126025558556403
Recall: 0.8045273549415425
Ensemble accuracy: 0.8043785310734464
F1 Score: 0.8071886614976552
Confusion Matrix:
[[365  90   5]
 [ 70 390  31]
 [ 19  62 384]]


In [24]:
probabilities = dt_meta_classifier.predict_proba(test_meta_features)

# Calculate precision-recall curve and find optimal thresholds for each class
dt_thresh = []
for i in range(probabilities.shape[1]):  # Iterate over each class
    precision, recall, thresholds = precision_recall_curve(y_test == i, probabilities[:, i])
    f1_scores = 2 * (precision * recall) / (precision + recall)
    optimal_idx = np.nanargmax(f1_scores)  # Handle cases where precision and recall are both zero
    dt_thresh.append(thresholds[optimal_idx])

# Apply the optimized thresholds to make adjusted predictions
adjusted_predictions = np.array([np.argmax([probabilities[i, j] if probabilities[i, j] >= dt_thresh[j] else 0
                                         for j in range(probabilities.shape[1])])
                              for i in range(len(probabilities))])

# Evaluate
adjusted_precision = precision_score(y_test, adjusted_predictions, average='macro')
adjusted_recall = recall_score(y_test, adjusted_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_test, adjusted_predictions)
adjusted_f1 = f1_score(y_test, adjusted_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, adjusted_predictions)

print("Optimized thresholds for each class:", dt_thresh)
print("Decision Tree")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Ensemble accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1}")
print("Confusion Matrix:")
print(conf_matrix)

Optimized thresholds for each class: [0.359375, 0.4411764705882353, 0.5]
Decision Tree
Precision: 0.8126
Recall: 0.8045
Ensemble accuracy: 0.8044
F1 Score: 0.8071886614976552
Confusion Matrix:
[[365  90   5]
 [ 70 390  31]
 [ 19  62 384]]


In [25]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!B6:E6'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


### SVM

In [26]:
# Evaluate
precision = precision_score(y_test, svm_final_predictions, average='macro')
recall = recall_score(y_test, svm_final_predictions, average='macro')
accuracy = accuracy_score(y_test, svm_final_predictions)
f1 = f1_score(y_test, svm_final_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, svm_final_predictions)

print("SVM")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Ensemble accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


SVM
Precision: 0.8275921833701633
Recall: 0.8253564313478675
Ensemble accuracy: 0.8241525423728814
F1 Score: 0.826186768001253
Confusion Matrix:
[[394  65   1]
 [ 72 376  43]
 [  5  63 397]]


In [27]:
probabilities = svm_meta_classifier.predict_proba(test_meta_features)

# Calculate precision-recall curve and find optimal thresholds for each class
svm_thresh = []
for i in range(probabilities.shape[1]):  # Iterate over each class
    precision, recall, thresholds = precision_recall_curve(y_test == i, probabilities[:, i])
    f1_scores = 2 * (precision * recall) / (precision + recall)
    optimal_idx = np.nanargmax(f1_scores)  # Handle cases where precision and recall are both zero
    svm_thresh.append(thresholds[optimal_idx])

# Apply the optimized thresholds to make adjusted predictions
adjusted_predictions = np.array([np.argmax([probabilities[i, j] if probabilities[i, j] >= svm_thresh[j] else 0
                                         for j in range(probabilities.shape[1])])
                              for i in range(len(probabilities))])

# Evaluate
adjusted_precision = precision_score(y_test, adjusted_predictions, average='macro')
adjusted_recall = recall_score(y_test, adjusted_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_test, adjusted_predictions)
adjusted_f1 = f1_score(y_test, adjusted_predictions, average='macro')
conf_matrix = confusion_matrix(y_test, adjusted_predictions)

print("Optimized thresholds for each class:", svm_thresh)
print("SVM")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Ensemble accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1}")
print("Confusion Matrix:")
print(conf_matrix)

Optimized thresholds for each class: [0.14522214373579767, 0.42119295394355516, 0.5100039263429087]
SVM
Precision: 0.8201
Recall: 0.8022
Ensemble accuracy: 0.8030
F1 Score: 0.806287587411107
Confusion Matrix:
[[333 126   1]
 [ 42 406  43]
 [  5  62 398]]


In [28]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!B7:E7'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


## TEST DATA C

In [29]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load validation data
validation_file_path = 'Corpus/FiReCS/FiReCS_data_c.csv'
validation_df = pd.read_csv(validation_file_path)

X_val = validation_df['review'].values 
y_val = validation_df['label'].values

# Generate meta-features for data c dataset
val_meta_features = get_predictions(model_info, X_val, y_val, device) 

# Predict using the logistic regression meta-classifier
val_lr_predictions = lr_meta_classifier.predict(val_meta_features)

# Predict using the naive bayes meta-classifier
val_nb_predictions = nb_meta_classifier.predict(val_meta_features)

# Predict using the decision tree meta-classifier
val_dt_predictions = dt_meta_classifier.predict(val_meta_features)

# Predict using the svm meta-classifier
val_svm_predictions = svm_meta_classifier.predict(val_meta_features)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

### LR

In [30]:
# Evaluate the predictions
precision = precision_score(y_val, val_lr_predictions, average='macro')
recall = recall_score(y_val, val_lr_predictions, average='macro')
accuracy = accuracy_score(y_val, val_lr_predictions)
f1 = f1_score(y_val, val_lr_predictions, average='macro')
conf_matrix = confusion_matrix(y_val, val_lr_predictions)

print("Evaluation on HS-Data C Dataset")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Evaluation on HS-Data C Dataset
Precision: 0.8073078368959546
Recall: 0.8024954852599263
Accuracy: 0.8017159199237369
F1 Score: 0.8041957308433526
Confusion Matrix:
[[266  72   3]
 [ 48 276  40]
 [  2  43 299]]


In [31]:
lr_validation_probabilities = lr_meta_classifier.predict_proba(val_meta_features)

# Apply thresholds to the validation set probabilities to make final predictions
lr_validation_predictions = np.array([np.argmax([lr_validation_probabilities[i, j] if lr_validation_probabilities[i, j] >= lr_thresh[j] else 0
                                              for j in range(lr_validation_probabilities.shape[1])])
                                   for i in range(len(lr_validation_probabilities))])

# Evaluate threshold on validation set
adjusted_precision = precision_score(y_val, lr_validation_predictions, average='macro')
adjusted_recall = recall_score(y_val, lr_validation_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_val, lr_validation_predictions)
adjusted_f1 = f1_score(y_val, lr_validation_predictions, average='macro')
adjusted_conf_matrix = confusion_matrix(y_val, lr_validation_predictions)

# Print the evaluation results
print("Validation Results")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1:.4f}")
print("Confusion Matrix:")
print(adjusted_conf_matrix)


Validation Results
Precision: 0.8047
Recall: 0.8029
Accuracy: 0.8017
F1 Score: 0.8036
Confusion Matrix:
[[272  64   5]
 [ 54 270  40]
 [  3  42 299]]


In [32]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!F4:I4'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


#### Save CM

In [33]:
import altair as alt
import altair_saver
import pandas as pd
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_val, lr_validation_predictions)

# Define class names
class_names = ['Negative', 'Neutral', 'Positive']

# Convert confusion matrix to DataFrame
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names).reset_index().melt(id_vars='index')
cm_df.columns = ['True', 'Predicted', 'Count']

# Ensure the order of categories
cm_df['True'] = pd.Categorical(cm_df['True'], categories=class_names, ordered=True)
cm_df['Predicted'] = pd.Categorical(cm_df['Predicted'], categories=class_names, ordered=True)

# Create the Altair plot
heatmap = alt.Chart(cm_df).mark_rect().encode(
    x=alt.X('Predicted:O', sort=class_names),
    y=alt.Y('True:O', sort=class_names),
    color='Count:Q',
    tooltip=['True', 'Predicted', 'Count']
).properties(
    width=400,
    height=300,
    title='OF_Using_FIRE 4STK-1 LR'
)

# Add text labels
text = heatmap.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    text='Count:Q',
    color=alt.condition(
        alt.datum.Count > cm.max() / 2,
        alt.value('white'),
        alt.value('black')
    )
)

# Combine heatmap and text
final_chart = heatmap + text

# Display the plot
final_chart.show()

alt.LayerChart(...)

In [34]:
# Specify the folder path
folder_path = os.path.expanduser('Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/')

# Save the plot using vl-convert
file_path_png = os.path.join(folder_path, 'OF_Using_FIRE 4STK-1 LR.png')
final_chart.save(file_path_png)

print(f"Plot saved to {file_path_png}")

Plot saved to Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/OF_Using_FIRE 4STK-1 LR.png


### NB

In [35]:
# Evaluate the predictions
precision = precision_score(y_val, val_nb_predictions, average='macro')
recall = recall_score(y_val, val_nb_predictions, average='macro')
accuracy = accuracy_score(y_val, val_nb_predictions)
f1 = f1_score(y_val, val_nb_predictions, average='macro')
conf_matrix = confusion_matrix(y_val, val_nb_predictions)

print("Evaluation on HS-Data C Dataset")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Evaluation on HS-Data C Dataset
Precision: 0.8025095852213315
Recall: 0.7944900498351362
Accuracy: 0.7940896091515729
F1 Score: 0.7968316260429851
Confusion Matrix:
[[255  83   3]
 [ 45 279  40]
 [  2  43 299]]


In [36]:
nb_validation_probabilities = nb_meta_classifier.predict_proba(val_meta_features)

# Apply thresholds to the validation set probabilities to make final predictions
nb_validation_predictions = np.array([np.argmax([nb_validation_probabilities[i, j] if nb_validation_probabilities[i, j] >= nb_thresh[j] else 0
                                              for j in range(nb_validation_probabilities.shape[1])])
                                   for i in range(len(nb_validation_probabilities))])

# Evaluate threshold on validation set
adjusted_precision = precision_score(y_val, nb_validation_predictions, average='macro')
adjusted_recall = recall_score(y_val, nb_validation_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_val, nb_validation_predictions)
adjusted_f1 = f1_score(y_val, nb_validation_predictions, average='macro')
adjusted_conf_matrix = confusion_matrix(y_val, nb_validation_predictions)

# Print the evaluation results
print("Validation Results")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1:.4f}")
print("Confusion Matrix:")
print(adjusted_conf_matrix)


Validation Results
Precision: 0.8122
Recall: 0.8117
Accuracy: 0.8103
F1 Score: 0.8120
Confusion Matrix:
[[282  56   3]
 [ 55 269  40]
 [  3  42 299]]


In [37]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!F5:I5'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


#### Save CM

In [38]:
import altair as alt
import altair_saver
import pandas as pd
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_val, nb_validation_predictions)

# Define class names
class_names = ['Negative', 'Neutral', 'Positive']

# Convert confusion matrix to DataFrame
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names).reset_index().melt(id_vars='index')
cm_df.columns = ['True', 'Predicted', 'Count']

# Ensure the order of categories
cm_df['True'] = pd.Categorical(cm_df['True'], categories=class_names, ordered=True)
cm_df['Predicted'] = pd.Categorical(cm_df['Predicted'], categories=class_names, ordered=True)

# Create the Altair plot
heatmap = alt.Chart(cm_df).mark_rect().encode(
    x=alt.X('Predicted:O', sort=class_names),
    y=alt.Y('True:O', sort=class_names),
    color='Count:Q',
    tooltip=['True', 'Predicted', 'Count']
).properties(
    width=400,
    height=300,
    title='OF_Using_FIRE 4STK-1 NB'
)

# Add text labels
text = heatmap.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    text='Count:Q',
    color=alt.condition(
        alt.datum.Count > cm.max() / 2,
        alt.value('white'),
        alt.value('black')
    )
)

# Combine heatmap and text
final_chart = heatmap + text

# Display the plot
final_chart.show()

alt.LayerChart(...)

In [39]:
# Specify the folder path
folder_path = os.path.expanduser('Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/')

# Save the plot using vl-convert
file_path_png = os.path.join(folder_path, 'OF_Using_FIRE 4STK-1 NB.png')
final_chart.save(file_path_png)

print(f"Plot saved to {file_path_png}")

Plot saved to Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/OF_Using_FIRE 4STK-1 NB.png


### DT

In [40]:
# Evaluate the predictions
precision = precision_score(y_val, val_dt_predictions, average='macro')
recall = recall_score(y_val, val_dt_predictions, average='macro')
accuracy = accuracy_score(y_val, val_dt_predictions)
f1 = f1_score(y_val, val_dt_predictions, average='macro')
conf_matrix = confusion_matrix(y_val, val_dt_predictions)

print("Evaluation on HS-Data C Dataset")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Evaluation on HS-Data C Dataset
Precision: 0.7943999669004324
Recall: 0.7896344715549518
Accuracy: 0.7893231649189705
F1 Score: 0.7913948660345519
Confusion Matrix:
[[260  72   9]
 [ 53 280  31]
 [ 15  41 288]]


In [41]:
dt_validation_probabilities = dt_meta_classifier.predict_proba(val_meta_features)

# Apply thresholds to the validation set probabilities to make final predictions
dt_validation_predictions = np.array([np.argmax([dt_validation_probabilities[i, j] if dt_validation_probabilities[i, j] >= dt_thresh[j] else 0
                                              for j in range(dt_validation_probabilities.shape[1])])
                                   for i in range(len(dt_validation_probabilities))])

# Evaluate threshold on validation set
adjusted_precision = precision_score(y_val, dt_validation_predictions, average='macro')
adjusted_recall = recall_score(y_val, dt_validation_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_val, dt_validation_predictions)
adjusted_f1 = f1_score(y_val, dt_validation_predictions, average='macro')
adjusted_conf_matrix = confusion_matrix(y_val, dt_validation_predictions)

# Print the evaluation results
print("Validation Results")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1:.4f}")
print("Confusion Matrix:")
print(adjusted_conf_matrix)


Validation Results
Precision: 0.7944
Recall: 0.7896
Accuracy: 0.7893
F1 Score: 0.7914
Confusion Matrix:
[[260  72   9]
 [ 53 280  31]
 [ 15  41 288]]


In [42]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!F6:I6'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


#### Save CM

In [43]:
import altair as alt
import altair_saver
import pandas as pd
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_val, dt_validation_predictions)

# Define class names
class_names = ['Negative', 'Neutral', 'Positive']

# Convert confusion matrix to DataFrame
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names).reset_index().melt(id_vars='index')
cm_df.columns = ['True', 'Predicted', 'Count']

# Ensure the order of categories
cm_df['True'] = pd.Categorical(cm_df['True'], categories=class_names, ordered=True)
cm_df['Predicted'] = pd.Categorical(cm_df['Predicted'], categories=class_names, ordered=True)

# Create the Altair plot
heatmap = alt.Chart(cm_df).mark_rect().encode(
    x=alt.X('Predicted:O', sort=class_names),
    y=alt.Y('True:O', sort=class_names),
    color='Count:Q',
    tooltip=['True', 'Predicted', 'Count']
).properties(
    width=400,
    height=300,
    title='OF_Using_FIRE 4STK-1 DT'
)

# Add text labels
text = heatmap.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    text='Count:Q',
    color=alt.condition(
        alt.datum.Count > cm.max() / 2,
        alt.value('white'),
        alt.value('black')
    )
)

# Combine heatmap and text
final_chart = heatmap + text

# Display the plot
final_chart.show()

alt.LayerChart(...)

In [44]:
# Specify the folder path
folder_path = os.path.expanduser('Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/')

# Save the plot using vl-convert
file_path_png = os.path.join(folder_path, 'OF_Using_FIRE 4STK-1 DT.png')
final_chart.save(file_path_png)

print(f"Plot saved to {file_path_png}")

Plot saved to Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/OF_Using_FIRE 4STK-1 DT.png


### SVM

In [45]:
# Evaluate the predictions
precision = precision_score(y_val, val_svm_predictions, average='macro')
recall = recall_score(y_val, val_svm_predictions, average='macro')
accuracy = accuracy_score(y_val, val_svm_predictions)
f1 = f1_score(y_val, val_svm_predictions, average='macro')
conf_matrix = confusion_matrix(y_val, val_svm_predictions)

print("Evaluation on HS-Data C Dataset")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Evaluation on HS-Data C Dataset
Precision: 0.8169426066707034
Recall: 0.8131416907673472
Accuracy: 0.8122020972354623
F1 Score: 0.8147093849296737
Confusion Matrix:
[[277  62   2]
 [ 52 278  34]
 [  2  45 297]]


In [46]:
svm_validation_probabilities = svm_meta_classifier.predict_proba(val_meta_features)

# Apply thresholds to the validation set probabilities to make final predictions
svm_validation_predictions = np.array([np.argmax([svm_validation_probabilities[i, j] if svm_validation_probabilities[i, j] >= svm_thresh[j] else 0
                                              for j in range(svm_validation_probabilities.shape[1])])
                                   for i in range(len(svm_validation_probabilities))])

# Evaluate threshold on validation set
adjusted_precision = precision_score(y_val, svm_validation_predictions, average='macro')
adjusted_recall = recall_score(y_val, svm_validation_predictions, average='macro')
adjusted_accuracy = accuracy_score(y_val, svm_validation_predictions)
adjusted_f1 = f1_score(y_val, svm_validation_predictions, average='macro')
adjusted_conf_matrix = confusion_matrix(y_val, svm_validation_predictions)

# Print the evaluation results
print("Validation Results")
print(f"Precision: {adjusted_precision:.4f}")
print(f"Recall: {adjusted_recall:.4f}")
print(f"Accuracy: {adjusted_accuracy:.4f}")
print(f"F1 Score: {adjusted_f1:.4f}")
print("Confusion Matrix:")
print(adjusted_conf_matrix)


Validation Results
Precision: 0.8161
Recall: 0.7950
Accuracy: 0.7960
F1 Score: 0.7989
Confusion Matrix:
[[236 103   2]
 [ 28 302  34]
 [  3  44 297]]


In [47]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '4-1!F7:I7'  

values = [[
    f"{adjusted_precision * 100:.2f}",
    f"{adjusted_recall * 100:.2f}",
    f"{adjusted_accuracy * 100:.2f}",
    f"{adjusted_f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


#### Save CM

In [48]:
import altair as alt
import altair_saver
import pandas as pd
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_val, svm_validation_predictions)

# Define class names
class_names = ['Negative', 'Neutral', 'Positive']

# Convert confusion matrix to DataFrame
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names).reset_index().melt(id_vars='index')
cm_df.columns = ['True', 'Predicted', 'Count']

# Ensure the order of categories
cm_df['True'] = pd.Categorical(cm_df['True'], categories=class_names, ordered=True)
cm_df['Predicted'] = pd.Categorical(cm_df['Predicted'], categories=class_names, ordered=True)

# Create the Altair plot
heatmap = alt.Chart(cm_df).mark_rect().encode(
    x=alt.X('Predicted:O', sort=class_names),
    y=alt.Y('True:O', sort=class_names),
    color='Count:Q',
    tooltip=['True', 'Predicted', 'Count']
).properties(
    width=400,
    height=300,
    title='OF_Using_FIRE 4STK-1 SVM'
)

# Add text labels
text = heatmap.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    text='Count:Q',
    color=alt.condition(
        alt.datum.Count > cm.max() / 2,
        alt.value('white'),
        alt.value('black')
    )
)

# Combine heatmap and text
final_chart = heatmap + text

# Display the plot
final_chart.show()

alt.LayerChart(...)

In [49]:
# Specify the folder path
folder_path = os.path.expanduser('Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/')

# Save the plot using vl-convert
file_path_png = os.path.join(folder_path, 'OF_Using_FIRE 4STK-1 SVM.png')
final_chart.save(file_path_png)

print(f"Plot saved to {file_path_png}")

Plot saved to Results/Ensemble Model Results/On FireCS dataset/OF Using FIRE ENSEMBLE/Stacking/OF_Using_FIRE 4STK-1 SVM.png
